In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler



In [ ]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/Test'


In [ ]:
# Step 2: Extract HOG features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        hog_feature = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
        hog_features.append(hog_feature)
    # Reshape the hog_features to a 2D array
    return np.array(hog_features)  # Reshape to 2D



In [ ]:

# Function to load images and extract labels from the filenames
def load_images_from_folder(folder_path):
    images = []
    labels = []
    for image_name in os.listdir(folder_path):
        if image_name.endswith(('.jpg', '.png', '.jpeg')):  # Check for image file extensions
            img_path = os.path.join(folder_path, image_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
            img = cv2.resize(img, (128, 128))  # Resize to a consistent size
            images.append(img)

            # Extract label from the filename (assuming filenames like A+.jpg, B-.png)
            label = os.path.splitext(image_name)[0]  # Extract the part before the file extension
            labels.append(label)
    return np.array(images), np.array(labels)


# Now `images` contains the preprocessed images, and `labels` contains the corresponding blood type labels.
print("Loaded images:", len(images))
print("Labels:", labels)


Loaded images: 11
Labels: ['Bivep' 'ABivep' 'Aivep' 'Biven' 'Oivep' 'ABiven' 'Aiven' 'Oiven' 'A+'
 'A-' 'AB+']


In [ ]:
# Example usage: Predict blood type for an image
def predict_blood_type(image_path, model):
    # Load image and check if it loaded correctly
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Error: Could not read image from {image_path}")
        return None  # Or handle the error in a way that suits your application

    # Preprocess the image
    img_blur = cv2.GaussianBlur(img, (5, 5), 0)
    _, thresh = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5,5), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    resized_img = cv2.resize(morph, (128, 128))
    flat_img = resized_img.flatten()

    # Reshape and predict
    flat_img = flat_img.reshape(1, -1)
    prediction = model.predict(flat_img)
    return label_encoder.inverse_transform(prediction)[0]

In [ ]:

# Step 2: Preprocess images with thresholding and morphological operations
def preprocess_images(images):
    processed_images = []
    for img in images:
        # Apply GaussianBlur
        img_blur = cv2.GaussianBlur(img, (5, 5), 0)

        # Apply Otsu's thresholding
        _, thresh = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply morphological operations (closing)
        kernel = np.ones((5,5), np.uint8)
        morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        # Resize to a consistent size
        resized_img = cv2.resize(morph, (128, 128))

        # Flatten the image to create a feature vector
        flat_img = resized_img.flatten()

        processed_images.append(flat_img)
    return np.array(processed_images)

#Import LabelEncoder from sklearn
from sklearn.preprocessing import LabelEncoder

# Preprocess the images
processed_images = preprocess_images(images)

# Step 3: Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Step 4: Define the model pipeline
model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))

# Step 5: Train the model
model.fit(processed_images, encoded_labels)

# Step 6: Train the model on the entire dataset
model.fit(hog_features, encoded_labels)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(kernel='linear'))])

In [ ]:
predictions = model.predict(processed_images)

# Convert predictions back to original labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Calculate accuracy (since we have only 8 images, this is just a basic check)
accuracy = accuracy_score(encoded_labels, predictions)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

ValueError: X has 16384 features, but StandardScaler is expecting 8100 features as input.

In [ ]:
# Step 7: Predict on a new image
def predict_blood_type(image_path, model):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    hog_feature = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    hog_feature = hog_feature.reshape(1, -1)  # Reshape to ensure it is 2D (1 sample, n_features)
    prediction = model.predict(hog_feature)
    return label_encoder.inverse_transform(prediction)[0]


In [ ]:
test_image_path = '/content/drive/MyDrive/Colab Notebooks/Test/Bivep.jpg'
predicted_label = predict_blood_type(test_image_path, model)
print(f"The predicted blood type is: {predicted_label}")

The predicted blood type is: Bivep


In [ ]:
test_image_path = '/content/drive/MyDrive/Colab Notebooks/Test/A+.jpg'
predicted_label = predict_blood_type(test_image_path, model)
print(f"The predicted blood type is: {predicted_label}")

The predicted blood type is: A+


In [ ]:
test_image_path = '/content/drive/MyDrive/Colab Notebooks/Test/Oiven.jpg'
predicted_label = predict_blood_type(test_image_path, model)
print(f"The predicted blood type is: {predicted_label}")

The predicted blood type is: Oiven


In [ ]:
test_image_path = '/content/drive/MyDrive/Colab Notebooks/Test/A-.jpg'
predicted_label = predict_blood_type(test_image_path, model)
print(f"The predicted blood type is: {predicted_label}")

The predicted blood type is: A-


In [ ]:
test_image_path = '/content/drive/MyDrive/Colab Notebooks/Test/AB+.jpg'
predicted_label = predict_blood_type(test_image_path, model)
print(f"The predicted blood type is: {predicted_label}")

The predicted blood type is: AB+
